In [2]:
import pandas as pd

df_train = pd.read_csv('./train_full.csv')
df_validation = pd.read_csv('./validation_full.csv')

In [3]:
X_train, X_val, y_train, y_val = df_train.iloc[:,0:-1], df_validation.iloc[:,0:-1], df_train['label'].values*1, df_validation['label'].values*1

In [14]:
y_train = [10 if x==11 else x for x in y_train]
y_val = [10 if x==11 else x for x in y_val]

In [15]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_train_enc = np_utils.to_categorical(y_train_enc)
y_val_enc = encoder.fit_transform(y_val)
y_val_enc = np_utils.to_categorical(y_val)

In [64]:
from keras.layers import Dropout

def baseline_model():
  model = Sequential()
  model.add(Dense(2048, input_dim=2925, kernel_initializer='normal', activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(1024, kernel_initializer='normal', activation='relu'))
  model.add(Dense(512, kernel_initializer='normal', activation='relu'))
  model.add(Dense(128, kernel_initializer='normal', activation='relu'))
  model.add(Dense(11, kernel_initializer='normal', activation='softmax'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [57]:
# kfold = KFold(n_splits=5, shuffle=True, random_state=5555)
# estimator = KerasClassifier(build_fn=baseline_model, epochs=15, batch_size=100, verbose=1 ,)
# results = cross_val_score(estimator, X_train, y_train_enc, cv=kfold)
# print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [65]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=15, batch_size=100, verbose=1 ,)
estimator.fit(X_train, y_train_enc)

Epoch 1/15
51157/51157 [==============================] - 12s 241us/step - loss: 1.1520 - acc: 0.6758
Epoch 2/15
51157/51157 [==============================] - 12s 226us/step - loss: 0.7240 - acc: 0.7711
Epoch 3/15
51157/51157 [==============================] - 11s 224us/step - loss: 0.5815 - acc: 0.8153
Epoch 4/15
51157/51157 [==============================] - 11s 222us/step - loss: 0.5002 - acc: 0.8406
Epoch 5/15
51157/51157 [==============================] - 11s 222us/step - loss: 0.4520 - acc: 0.8565
Epoch 6/15
51157/51157 [==============================] - 11s 223us/step - loss: 0.4042 - acc: 0.8708
Epoch 7/15
51157/51157 [==============================] - 11s 220us/step - loss: 0.3770 - acc: 0.8801
Epoch 8/15
51157/51157 [==============================] - 12s 229us/step - loss: 0.3489 - acc: 0.8873
Epoch 9/15
51157/51157 [==============================] - 12s 227us/step - loss: 0.3239 - acc: 0.8964
Epoch 10/15
51157/51157 [==============================] - 11s 221us/step - loss: 

In [66]:
train_pred = estimator.predict(X_train)
val_pred = estimator.predict(X_val)
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
print('Train Class Accuracy:' + str(accuracy_score(y_train, train_pred)))
print('Test Class Accuracy:' + str(accuracy_score(y_val, val_pred)))
pd.DataFrame(confusion_matrix(y_val, val_pred))

5895/5895 [==============================] - 0s 73us/step
Train Class Accuracy:0.952479621557
Test Class Accuracy:0.875996607294


,0,1,2,3,4,5,6,7,8,9,10
0,204,2,0,0,3,0,0,0,0,0,13
1,2,147,5,3,2,1,0,1,0,16,49
2,0,1,167,0,5,0,1,5,1,2,28
3,0,13,1,162,1,0,4,0,0,3,54
4,14,2,1,0,179,6,0,1,0,0,20
5,0,0,0,0,6,176,0,1,0,0,36
6,0,0,3,1,0,0,154,8,0,1,58
7,1,0,13,0,2,0,1,182,0,0,21
8,0,0,8,0,0,0,0,0,177,0,28
9,0,38,7,8,0,0,1,0,0,119,46


In [44]:
pd.DataFrame(confusion_matrix(y_train, train_pred))

,0,1,2,3,4,5,6,7,8,9,10
0,1789,0,0,0,17,0,0,1,0,0,87
1,1,1620,0,13,1,0,0,1,0,55,136
2,0,1,1525,0,0,0,7,72,17,5,177
3,0,20,0,1655,0,0,4,0,0,6,178
4,6,0,0,0,1753,3,0,19,0,0,120
5,0,0,0,0,14,1634,0,1,0,0,227
6,0,0,0,5,0,0,1670,7,0,0,158
7,0,0,5,0,0,1,19,1730,0,0,106
8,0,1,1,0,1,0,0,2,1784,0,127
9,0,119,3,29,0,0,2,0,1,1471,198
